# Entrenamiento de una neurona simple

Vamos a comenzar por implementar una neurona simple definiendola con los parametros que hemos visto.


### Neurona simple


In [ ]:
import numpy as np 

Definimos la función de activación, por ejemplo una sigmoide

In [ ]:
def sigmoid(x):
  # Our activation function: f(x) = 1 / (1 + e^(-x))
  return ###

Definimos una clase que sea nuestra neurona

In [ ]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = ##
    self.bias = ##

  def feedforward(self, inputs):
    # Weight inputs, add bias, then use the activation function
    total = ##
    return sigmoid(total)

Probamos lo que hemos hecho hasta ahora

In [ ]:
weights = np.array([0, 1]) 
bias = 4
n = Neuron(weights, bias)

x = np.array([2, 3])      
print(n.feedforward(x)) 

0.9990889488055994


## Entrenamiento de una red neuronal

Vamos a proceder a entrenar ahora una red neuronal.


### Los datos 
Vamos a usar unos datos dummies para ver como funciona la red.

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples = 1000, noise=0.2, random_state=1993)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1993)


### Inicialización de las capas de la red neuronal

Lo primero que vamos a hacer, como en cualquier entrenamiento de redes neuronales, es llevar a cabo una inicialización que nos sirva de punto de partida para los pesos de las neuronas y sus bias.

Para este cometido, vamos a emplear un vector de diccionarios con la información de cada una de las capas de la red 

In [ ]:
X.shape[0]

10000

In [ ]:
nn_architecture = [
    {"input_dim": 2, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

Aqui podemos observar como en la definición de los parámetros se debe respetar el rango de dimension de la entrada de una capa con la salida de la anterior.

¿Cuántas entradas aceptará nuestra red?

Procedemos ahora a utilizar esta información para inicializar las capas

In [ ]:
def inicializar_capas(nn_architecture, seed = 1993):
    np.random.seed(seed)
    number_of_layers = ###
    parametros = ###

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        parametros['W' + str(layer_idx)] = ###
        parametros['b' + str(layer_idx)] = ###
        
    return parametros

¿Por qué esta inicialización? Por la ruptura de la simetría

### Definición de la función de activación
Vamos a proceder ahora a definir distintas funciones de activación que hemos visto durante la clase pero ahora en formato código 

In [ ]:
def sigmoid(Z):
  sigmoid_func = ###
  return sigmoid_func

def relu(Z):
  relu_func = ###
  return relu_func

Para poder hacer en los pasos de Backpropagation o actualización de parametros, calcularemos las funciones que nos devuelvan estos valores.

Estos valores osn la derivada de la función de activación multiplicada por el incremento de cambio



In [ ]:
def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

### Cálculo del Forward propagation
Este paso sería el cálculo de la salida de nuestra red neuronal, igual que hemos visto en clase pero esta vez aplicado a multiples capas.

Lo primero que vamos a hacer es definir el comportamiento interno de cada una de las capas que tendremos.

In [ ]:
def single_layer_forward_propagation(A_prev,
                                     W_act, 
                                     b_act, 
                                     activation="relu"):
  

  '''' Definimos una funcion que haga el calculo de cada una de las 
  neuronas que compone a las neuronas de una capa
  '''
  Z_act = np.dot(W_act, A_prev) + b_act
  
  if activation is "relu":
      funcion_de_activacion = ###
  elif activation is "sigmoid":
      funcion_de_activacion = ###

  A_act = ###
  return A_act, Z_act

Con una capa construida, ya podemos construir el total de la red utilizando la función que hemos creado previamente

In [ ]:
def full_forward_propagation(X, 
                             params_values, 
                             nn_architecture):
    memory = {}
    A_act = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = ###
        A_prev = ###
        
        activ_function_act = ###
        W_act = ###
        b_act = ###
        A_act, Z_act = single_layer_forward_propagation(A_prev, 
                                                        W_act, 
                                                        b_act, 
                                                        activ_function_act)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_act
       
    return A_act, memory

### Función de perdida
Como hemos visto en clase, es necesario definir una función de perdida para poder llevar a cabo el descenso del graciente y que en cada iteración se evalue.

Como en este caso estamos haciendo un problema de clasificación usaremos la CrossEntropy
![picture](https://drive.google.com/uc?id=1AiroSwCaqDt9tOyaLd0iX8v1kt23UChC)

In [ ]:
def get_cost_value(Y_pred, Y):
    m = Y_pred.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_pred).T) + np.dot(1 - Y, np.log(1 - Y_pred).T))
    return np.squeeze(cost)

### Calculo del Backward Propagation

Para actualizar los pesos y bias de las neuronas de nuestra red, hemos dicho que es necesario llevar a cabo un "camino hacia atras" que nos permita actualizar estos parametros desde las ultimas capas a las primeras mediante el uso de la derivada del gradiente parcial por cada una de las variables que componen a la red.

![picture](https://drive.google.com/uc?id=1Z3idGhAIAk8PoDktTxg4IoZtEax9oCDM)

Vamos a programar el paso en cada una de las capas para que sea más sencillo su entendimiento.

In [ ]:
def single_layer_backward_propagation(dA_act,
                                      W_act,
                                      b_act,
                                      Z_act,
                                      A_prev,
                                      f_act="relu"):
    m = A_prev.shape[1]
    if f_act is "relu":
        backward_activation_func = relu_backward
    elif f_act is "sigmoid":
        backward_activation_func = sigmoid_backward
    
    dZ_act = backward_activation_func(dA_act, Z_act)
    dW_act = np.dot(dZ_act, A_prev.T) / m
    db_act = np.sum(dZ_act, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_act.T, dZ_act)

    return dA_prev, dW_act, db_act

Y ahora lo aplicamos para todas las capas 

In [ ]:
def full_backward_propagation(Y_hat, 
                              Y, 
                              memory, 
                              params_values, 
                              nn_architecture):
    grads_values = {}
    m = Y.shape[0]
    Y = Y.reshape(Y_hat.shape)
   
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_act = layer_idx_prev + 1
        activ_function_act = layer["activation"]
        
        dA_act = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_act = memory["Z" + str(layer_idx_act)]
        W_act = params_values["W" + str(layer_idx_act)]
        b_act = params_values["b" + str(layer_idx_act)]
        
        dA_prev, dW_act, db_act = single_layer_backward_propagation(
            dA_act, W_act, b_act, Z_act, A_prev, activ_function_act)
        
        grads_values["dW" + str(layer_idx_act)] = dW_act
        grads_values["db" + str(layer_idx_act)] = db_act
    
    return grads_values

### Actualización de los parametros

Una vez que tenmos los valores que se modifican los parametros de nuestra red los actualizamos teniendo en cuenta un coeficiente que ya conocemos, el learning rate.

![picture](https://drive.google.com/uc?id=1FmeaacqxB9hnzplZ-3Nn7seZkMVzk8rH)

In [ ]:
def actualizar_parametros(parametros, 
                          grads_values,
                          nn_architecture,
                          learning_rate):
    for layer_idx, layer in enumerate(nn_architecture, 1):
        parametros["W" + str(layer_idx)] -= ###   
        parametros["b" + str(layer_idx)] -= ###

    return parametros

### Unificamos todos los pasos para el entrenamiento

Ponemos ahora todo en común para obtener el entrenamiento de nuestra red.

In [ ]:
def train(X, Y, 
          nn_architecture, 
          epochs, 
          learning_rate):
  
    parametros = inicializar_capas(nn_architecture, 2)
    cost_history = []    

    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, parametros, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)    

        grads_values = full_backward_propagation(Y_hat, Y, cashe, parametros, nn_architecture)
        parametros = actualizar_parametros(parametros, grads_values, nn_architecture, learning_rate)
      
    return parametros, cost_history

### Entrenamos y comprobamos rendimiento

Obtenemos los parametros obtenidos de nuestro entrenamiento

In [ ]:
train_param = train(np.transpose(X_train), 
                      np.transpose(y_train.reshape((y_train.shape[0], 1))),
                      nn_architecture, 10000, 0.01)[0]

Calculamos con forward propagation la salida de nuestro modelo para los test_values

In [ ]:
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), train_param, nn_architecture)

Obtenemos la accuracy del modelo. Para ello nos servimos de esta función.

In [ ]:
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

In [ ]:
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.reshape((y_test.shape[0], 1))))
print("Test set accuracy: {:.2f}".format(acc_test))



Test set accuracy: 0.96
